<a href="https://colab.research.google.com/github/soosung80/capstone-210-fall2022/blob/main/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.0 MB/s 
     |████████████████████████████████| 7.6 MB 33.7 MB/s 
     |████████████████████████████████| 182 kB 56.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 7.3 MB/s 


In [ ]:
from io import BytesIO
import requests
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc, roc_auc_score
from geopy.geocoders import Nominatim
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
import transformers
from transformers import BloomForCausalLM
from transformers import BloomForTokenClassification
from transformers import BloomForTokenClassification
from transformers import BloomTokenizerFast
import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
r = requests.get('https://docs.google.com/spreadsheets/d/e/2PACX-1vQQs7i_LXcx3hFcJKUNZmVEnwDJ49vwJGE02Lpv5Em-N9Vosfpp7vKIzmKg5-bCjtbh1ZHV1hnGEvSN/pub?gid=872031091&single=true&output=csv')
data = r.content
df = pd.read_csv(BytesIO(data), index_col=0)
df.head()

,Item name,Category,Price,Unit,Unit Price,Store,Date,Product Number,Sale Price,Serving Size,...,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrate (g),Dietary Fiber (g),Total Sugars (g),Added Sugars (g),Protein (g)
0,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100 Calorie Greek Yogurt (Tropical ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100 Calorie Greek Yogurt (Blackberry),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100 Calorie Greek Yogurt (Strawberry),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aged Cheddar (Four Year),dairy,3.69,$/oz,0.53,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(df)

8630

In [ ]:
r1=requests.get('https://docs.google.com/spreadsheets/d/e/2PACX-1vRiMOi8VpJPX1m6VXIsMbW8S7IPhs6igS-PNkrMKwAhGvjp57LWUQejM3PCAowbSjNTDcrQzh0mxCbM/pub?gid=2106582341&single=true&output=csv')
store_df = pd.read_csv(BytesIO(r1.content))
store_df.head(2)

,Borough,County,Operation Type,Entity Name,Street Number,Street Name,City,State,Zip Code,Coordinates
0,Manhattan,New York,Store,WHOLE FOODS MARKET GROUP INC,450,W 33RD ST,NEW YORK,NY,10001,"(40.75294109055737, -73.99817016423056)"
1,Manhattan,New York,Store,WHOLE FOODS MARKET GROUP INC,250,7TH AVE,NEW YORK,NY,10001,"(40.74667173490512, -73.99570174531223)"


In [ ]:
store_df['Store']=store_df['Entity Name'].map({'TRADER JOES EAST INC':"Trader Joe's",
                                               'ALDI INC':'Aldi',
                                               'WHOLE FOODS MARKET GROUP INC':'Wholefoods Market',
                                               'FOOD TOWN': 'Foodtown Market',
                                               'FAIRWAY MARKET':'Fairway Market' })
df['Store']=df['Store'].replace(regex='Footdown', value='Foodtown')

In [ ]:
store_df['Store'].unique()

array(['Wholefoods Market', "Trader Joe's", 'Fairway Market', 'Aldi',
       'Foodtown Market'], dtype=object)

In [ ]:
df['Store'].unique()

array(['Aldi', "Trader Joe's", 'Fairway Market', 'Foodtown Market',
       'Wholefoods Market'], dtype=object)

# Location calculation

In [ ]:
def get_coordinates(zipcode):
  #Using Nominatim Api
  geolocator = Nominatim()
  location = geolocator.geocode(zipcode, timeout=100000)
  return location.latitude, location.longitude


#   #   # install
# # !pip install geopy
# # # import
# # from geopy.geocoders import Nominatim
# # # code
# app = Nominatim()
# # address = "First St SE, Washington, DC 20004, United States"
# address='450 W thirty third ST, NEW YORK, NY 10001, United States'
# location = app.geocode(address).raw
# print(location)

# print(location['lat'], location['lon'])
# # # >>> ('38.8716329', '-77.005766')


In [ ]:
# create column coordinates
store_df['Street']=store_df[['Street Number', 'Street Name']].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)

store_df['Address']=store_df[['Street', 'City', 'State', 'Zip Code']].apply(lambda x: ', '.join(x.dropna().astype(str)),axis=1)


In [ ]:
store_df['Coordinates']=store_df['Zip Code'].apply(get_coordinates)


/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [ ]:
#create new dataframe store locator
store_locator=store_df[['Borough','Store', 'Address', 'Zip Code','Coordinates']].drop_duplicates()

#merge dataframes
merged_df=df.merge(store_locator, how='left', on='Store')
merged_df.head(3)

,Item name,Category,Price,Unit,Unit Price,Store,Date,Product Number,Sale Price,Serving Size,...,Sodium (mg),Total Carbohydrate (g),Dietary Fiber (g),Total Sugars (g),Added Sugars (g),Protein (g),Borough,Address,Zip Code,Coordinates
0,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,"5532 BROADWAY, BRONX, NY, 10463",10463,"(40.88099069767225, -73.90565087656735)"
1,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,"1628 FLATBUSH AVE, BROOKLYN, NY, 11210",11210,"(40.62741664063716, -73.94567882204815)"
2,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,"528 GATEWAY DR, BROOKLYN, NY, 11239",11239,"(40.649454134010156, -73.8827212147208)"


In [ ]:
store_df.count()

Borough           61
County            61
Operation Type    61
Entity Name       61
Street Number     61
Street Name       61
City              61
State             61
Zip Code          61
Coordinates       61
Store             61
Street            61
Address           61
dtype: int64

In [ ]:
import geopy.distance

users_zipcode=10128
merged_df=merged_df.assign(User_coordinates = get_coordinates(users_zipcode))

def distancer(row):
    coords_1 = (row['Coordinates'])
    coords_2 = (row['User_coordinates'])
    return geopy.distance.VincentyDistance(coords_1, coords_2).miles


merged_df['Distance'] = merged_df.apply(distancer, axis=1)


def filter_distance(user_mile_distance=None):
  if user_mile_distance is None:

    df=merged_df[merged_df['Distance'] <= 5]

  else: 

    df=merged_df[merged_df['Distance'] <= user_mile_distance]

  return df

ValueError: ignored

#Create Dietary Restriction columns

In [ ]:
df['ItemName']= df['Item name'].str.replace(',', ' ').str.replace('-', ' ').str.lower()

df['Kosher'] = np.where( df['ItemName'].str.contains('kosher')==True, 1, 0)
# df['Dairy Free'] = np.where( (df['Item name'].str.contains('dairy free')==True) &
#                             (df['Item name'].str.contains('almond milk')==True), 1, 0)
# df['Gluten Free'] = np.where( df['ItemName'].str.contains('gluten free')==True, 1, 0)

## Write out updated dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
merged_df.head()
merged_df.to_csv('/content/gdrive/MyDrive/capstone project - shared/datasets/updated_df.csv')

#Creating more defined column ' Item'

In [ ]:
item_names={
  'cod':'Cod','cab':'Cab','cockles': 'Cockles','caddock':'Haddock','figs':'Figs','lime':'Lime','fluke': 'Fluke','mushrooms':'Mushrooms',
  'tuna':'Tuna','trout':'Trout','tilapia':'Tilapia','salmon':'Salmon','oysters':'Oysters','sardines':'Sardines','shrimp':'Shrimp',
  'scallops': 'Scallops','lobster':'Lobster','fillet':'Fillet', 'Perch':'Perch','Seafood Boil': 'Seafood Boil',
  'seafood mix': 'Seafood Mix','mackerel':'Mackerel','mussels':'Mussels','crab':'Crab','swordfish':'Swordfish','sea bass':'Sea Bass',
  'branzini':'Branzini','steak': 'Steak','lemons':'Lemon','avocado':'Avocado','banana':'Banana','cucumber':'Cucumber', 'blackberries':'Blackberries',
  'porgie':'Porgie','herring':'Herring','yogurt': 'yogurt','Yoghurt': 'Yogurt','egg':'Egg','juice':'Juice','butter':'Butter',
  'cheddar':'Cheese','cheese':'Cheese','brie':'Cheese','feta': 'Cheese','mozzarella':'Cheese','parmesan':'Cheese','parmigiano':'Cheese',
  'American imitation singles':'Cheese','milk':'Milk','nog':'Nog','smoothie':'Smoothie','bacon':'Bacon','apples':'Apples','pumpkin':'Pumpkin',
  'beans':'Beans','tomatoes':'Tomatoes','corn':'Corn','cantaloupe':'Cantaloupe','blueberries':'Blueberries','pear': 'Pear','peaches': 'Peaches',
  'grapes':'Grapes','cherries':'Cherries','clementines':'Oranges','oregano':'Oregano','kiwi':'Kiwi','oranges': 'Oranges','mango':'Mango','grapefruit':'Grapefruit',
  'cranberries': 'Cranberries','papaya':'Papaya','pineapple':'Pineapple','plum':'Plum','carrot':'Carrot','lettuce':'Lettuce','cabbage': 'Cabbage',
  'oil':'Oil','sour cream':'Sour Cream','strawberries':'Strawberries','watermelon':'Watermelon','salad':'Salad','potato':'Potato','broccoli':'Broccoli',
  'onion':'Onion','spinach':'Spinach', 'zucchini' : 'Zucchini','pepperoni': 'Pepperoni','mandarin':'Orange','garlic':'Garlic','duck' : 'Duck','pork' : 'Pork',
  'fish' : 'Fish','Half & Half': 'Milk','scallop' : 'Scallop','Anchovies': 'Anchovies','whipping cream': 'Whipping cream','whipped': 'Whipping cream',
  'gouda': 'Cheese','dates': 'Dates','nectarines' : 'Nectarines','pomegranates' : 'Pomegranates','prunes' : 'Prunes','raspberries':'Raspberries','beef':'Beef',
  'salami' : 'Salami','chicken':'Chicken','turkey': 'Turkey','lamb' : 'Lamb','ham':'Ham','beer':'Beer', 'seitan': 'Seitan','hot dogs': 'Hot Dogs',
  'prosciutto': 'Prosciutto','bratwurst':'Bratwurst','tofu':'Tofu','falafel': 'Falafel','sausage': 'Sausage','kefir':'Kefir','bass': 'Bass','whitefish' : 'Whitefish',
  'sauce':'Sauce','mahi mahi': 'Mahi Mahi','pollock': 'Pollock','whiting': 'Whiting','carp': 'Carp','kale':'Kale','arugula': 'Arugula','celery':'Celery',
  'ginger': 'Ginger','squid':'Squid','seafood blend': 'Seafood Blend','brussels sprouts': 'Brussels Sprouts','pike':'Pike', 'chocolate':'Chocolate','haddock': 'Haddock',
  'green peppers': 'Green Peppers','jalapeno peppers':'Jalapeno Peppers','almonds': 'Almonds','peanuts':'Peanuts','cauliflower':'Cauliflower','snapper': 'Snapper',
  'sweet peppers': 'Sweet Peppers','red bell pepper': 'Red Bell Pepper', 'radish': 'Radish','yams':'Yams','dumplings': 'Dumplings','cilantro': 'Cilantro',
  'plantain':'Plantain','shark': 'Shark', 'collard greens': 'Collard Greens','mushroom':'Mushroom','artichoke': 'Artichoke', 'spaghetti':'pasta','squash':'squash', 
  'buffalo wings': 'Buffalo Wings', 'meatballs':'Meatballs','proscuitto':'Proscuitto', 'veal':'Veal', 'tempeh':'Tempeh', 'crumbles':'Crumbles','nuggets':'Nuggets',
  'burger':'Burger Patties'
  }
def func(dictionary, col):
  for k, v in dictionary.items():
    if k in col:
      return v
df['Item']=df['Item name'].map(lambda x: func(item_names, x))
it=list(df['Item'].unique())
it = [i for i in it if i is not None]

# list(df.query("Item not in @it & Category in ['meat']")['Item name'])
#list(df.query("Item not in @it")['Item name'])
            

# KNN Classification

In [ ]:
new_df=df.dropna(subset=['Item'])

categorical_features= ["Store","Item name"]
numerical_features= ["Price"]

X = new_df[categorical_features + numerical_features]
y=new_df['Item'].values

# # Split dataset into random train and test subsets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)


In [ ]:
# Standardize features by removing mean and scaling to unit variance:
le = preprocessing.LabelEncoder()
X_train['Store'] = le.fit_transform(X_train['Store'])
X_train['Item name'] = le.fit_transform(X_train['Item name'])

X_test['Store'] = le.fit_transform(X_test['Store'])
X_test['Item name'] = le.fit_transform(X_test['Item name'])

# # Training the K-NN model on the Training set
for k in [3,5,6,10,15,20,25, 30,35]:
  classifier = KNeighborsClassifier(n_neighbors = k)
  classifier.fit(X_train, y_train)

  # # # Predicting the Test set results
  y_pred = classifier.predict(X_test)

  # # # Making the Confusion Matrix
  from sklearn.metrics import confusion_matrix, accuracy_score
  cm = confusion_matrix(y_test, y_pred)
  ac = accuracy_score(y_test, y_pred)
  print({k:ac})
  ac

{3: 0.19444444444444445}
{5: 0.26851851851851855}
{6: 0.3055555555555556}
{10: 0.3055555555555556}
{15: 0.32407407407407407}
{20: 0.3333333333333333}
{25: 0.32407407407407407}
{30: 0.3148148148148148}
{35: 0.32407407407407407}


#Gradient Boosting Model

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler

In [ ]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=3, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))

Learning rate:  0.05
Accuracy score (training): 0.779
Accuracy score (validation): 0.417
Learning rate:  0.075
Accuracy score (training): 0.807
Accuracy score (validation): 0.407
Learning rate:  0.1
Accuracy score (training): 0.863
Accuracy score (validation): 0.333
Learning rate:  0.25
Accuracy score (training): 0.968
Accuracy score (validation): 0.333
Learning rate:  0.5
Accuracy score (training): 0.791
Accuracy score (validation): 0.296
Learning rate:  0.75
Accuracy score (training): 0.867
Accuracy score (validation): 0.352
Learning rate:  1
Accuracy score (training): 0.012
Accuracy score (validation): 0.028


#NLP Search Engine

In [ ]:
#NLP Search Engine

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df_temp = merged_df[merged_df['Distance'] <= 3]
df_temp['metadata'] = df_temp.apply(lambda x : x['Item name'], axis = 1)
count_vec = CountVectorizer(stop_words='english')
count_vec_matrix = count_vec.fit_transform(df_temp['metadata'])


KeyError: ignored

In [ ]:
# This function takes in any words and vectorizes them, then finds similar vector in the count_vec_matrix

def vectorize_products_based_on_metadata(product_input):

    vec = count_vec.transform(pd.Series(product_input))
    
    simil = cosine_similarity(vec, count_vec_matrix)
    
    simil_scores = pd.DataFrame(simil.reshape(df_temp.shape[0],), index = df_temp.index, columns=['score'])
    
    # Don't return scores of zero, only as many positive scores as exist
    non_zero_scores = simil_scores[simil_scores['score'] > 0]
    
    if len(non_zero_scores) == 0:
        print('No similar products found.  Please refine your search terms and try again')
        return
    
    if len(non_zero_scores) < 10:
        item_count = len(non_zero_scores)
    else:
        item_count = len(non_zero_scores)
    
    similarity_scores = simil_scores.sort_values(['score'], ascending=False)[:10]
    
    return df_temp[df_temp.index.isin(list(similarity_scores.index))][['Item name','Unit Price','Unit','Store','Address', 'Distance']]



In [ ]:
count_vec_matrix.shape

In [ ]:
vectorize_products_based_on_metadata("Greek yogurt")

In [ ]:
vectorize_products_based_on_metadata("Greek yogurt")[vectorize_products_based_on_metadata("Greek yogurt")['Unit Price'] < 0.5]

## Subcategory Classification

### Clean up Item Name Column

In [72]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [73]:
#read in our dataset
ny_df = pd.read_csv('/content/gdrive/MyDrive/capstone project - shared/datasets/updated_df.csv')
ny_df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Item name,Category,Price,Unit,Unit Price,Store,Date,Product Number,Sale Price,...,Sodium (mg),Total Carbohydrate (g),Dietary Fiber (g),Total Sugars (g),Added Sugars (g),Protein (g),Borough,Address,Zip Code,Coordinates
0,0,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,"5532 BROADWAY, BRONX, NY, 10463",10463,"(40.88099069767225, -73.90565087656735)"
1,1,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,"1628 FLATBUSH AVE, BROOKLYN, NY, 11210",11210,"(40.62741664063716, -73.94567882204815)"
2,2,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,"528 GATEWAY DR, BROOKLYN, NY, 11239",11239,"(40.64924922731707, -73.8822178092683)"
3,3,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,"3006 THIRD AVE, BRONX, NY, 10455",10455,"(40.815460183861425, -73.90883523105197)"
4,4,100 Calorie Greek Yogurt (Blueberry ),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,"3785 NOSTRAND AVE, BROOKLYN, NY, 11235",11235,"(40.58620176091237, -73.95168128857959)"


In [74]:
#lower product name
ny_df['clean_item_name'] = ny_df["Item name"].str.lower()
ny_df['clean_item_name']

0                     100 calorie greek yogurt (blueberry )
1                     100 calorie greek yogurt (blueberry )
2                     100 calorie greek yogurt (blueberry )
3                     100 calorie greek yogurt (blueberry )
4                     100 calorie greek yogurt (blueberry )
                                ...                        
102270    uncooked - shell on frozen key west pink shrim...
102271    uncooked - shell on frozen key west pink shrim...
102272    uncooked - shell on frozen key west pink shrim...
102273    uncooked - shell on frozen key west pink shrim...
102274    uncooked - shell on frozen key west pink shrim...
Name: clean_item_name, Length: 102275, dtype: object

In [75]:
#remove unit amounts and stuff in parenthesis
import re

units = ['fl', 'lb','oz','fluidounces','fluid ounces', 'fl oz', 'per pound','grams']

regexp = re.compile(r'(\d+)\s*(%s)\b' % '|'.join(units))
# Function to clean the names
def Clean_names(item_name):
    # Search for opening bracket in the name followed by
    # any characters repeated any number of times
    if re.search('\,\s*\d+\s*.*', item_name):
        # Extract the position of beginning of pattern
        pos = re.search('\,\s*\d.*', item_name).start()
  
        # return the cleaned name
        return item_name[:pos]
    elif re.search('\(\s*.*\s*\).*',item_name):
       # Extract the position of beginning of pattern
        start = re.search('\(.*', item_name).start()
        end = re.search('\)', item_name).start()
        return item_name[:start]
  
    else:
        # if clean up needed return the same name
        return item_name
          
ny_df['clean_item_name'] =ny_df['clean_item_name'].apply(Clean_names)
ny_df[['Item name','clean_item_name']].sample(10)

,Item name,clean_item_name
73433,"Organic Carrot & Celery Sticks,14 OZ",organic carrot & celery sticks
55162,"Kiwi Fruit,1 CT",kiwi fruit
46198,"Friendship Cottage Cheese, Nonfat,16 OZ","friendship cottage cheese, nonfat"
14046,"Pre 95% Lean 5% Fat, Ground Beef,16.0 OUNCE","pre 95% lean 5% fat, ground beef"
86937,"Whipped Cream Cheese,8 OZ",whipped cream cheese
16546,"New York State Yellow Cheddar Cheese BLOCK,0.8...",new york state yellow cheddar cheese block
90284,"Part Skim Ricotta Cheese,16 OZ",part skim ricotta cheese
44013,"Miyoko's Organic Cream Cheese, Classic Vegan,8 OZ","miyoko's organic cream cheese, classic vegan"
54160,"Great American Raw Shrimp 16/20 Count,16 OZ",great american raw shrimp 16/20 count
47713,Pork Loin Rib End Chops Family Pack on the Bon...,pork loin rib end chops family pack on the bon...


In [76]:
#remove punctuation
import string

# create function for punctuation removal:
def remove_punctuations(text):
    for char in string.punctuation:
        text = text.replace(char, '')
    return text

# apply the function:
ny_df['clean_item_name'] =ny_df['clean_item_name'].apply(remove_punctuations)
ny_df[['Item name','clean_item_name']].sample(10)

,Item name,clean_item_name
71339,"Organic Dandelion Green,1 BUNCH",organic dandelion green
79168,"Cultured Lowfat Organic Buttermilk,946 ML",cultured lowfat organic buttermilk
74285,"baby bok choy,4 OZ",baby bok choy
65623,"Del Monte Sliced Peaches in Light Syrup,20 OZ",del monte sliced peaches in light syrup
82385,"siggiâ€™sÂ® Icelandic Skyr Nonfat Yogurt,5.3 OZ",siggiâ€™sâ® icelandic skyr nonfat yogurt
21509,"Pillsbury Crescents Rolls,8.0 OUNCE",pillsbury crescents rolls
25105,"Lactaid 100% Lactose Free Low Fat 1% Milk,96 FOZ",lactaid 100 lactose free low fat 1 milk
77890,"Cage Free Extra Large Brown Grade A Eggs,27 OZ",cage free extra large brown grade a eggs
40940,"Simply Oat Original,46 FOZ",simply oat original
48334,"Previously Frozen Steer Feet,LB",previously frozen steer feetlb


In [77]:
ny_df.clean_item_name.unique()

array(['100 calorie greek yogurt ', 'aged cheddar ',
       'alcohol infused cheddars ', ...,
       'boneless  skinless wild alaska pink salmon', 'tuna in olive oil',
       'uncooked  shell on frozen key west pink shrimp'], dtype=object)

### Subcategorize

In [78]:
ny_df.Category.unique()

array(['dairy', 'produce', 'meat', 'seafood'], dtype=object)

In [92]:
# create subcategory key words dictionary
# Format:   {'category':{'subcategory':['key words']}}
subcat_key_words = {'dairy':{ 'dairy substitutes':['soy','dairy-free','almond','lactaid','coconut','oat','nut milk','silk','dairy free','plant based','plantbased','almondmilk','lactose free','lactosefree','soymilk','oatmilk','cashewmilk'],
                            'yogurt':['yogurt','danimals','kefir','yoghurt','chobani','greek','smoothie'],
                             'milk products':['half','smoothie','2%','beverage','vanilla','cacao','mocha','nog','egg nog','milk','soymilk','buttermilk','half and half','half half','creamer','cream','heavy whipping cream','french vanilla'],
                             'cheese':['cheese','piave','italian blend','grand cru','caramella','mozz','fromage','humboldt fog','saint angel','kefalograviera','ovalie cendree','chevre','pepper jack','cheddar','mahon','daffinois','queso','sargento','havarti','manchego','asiago ','mascarpone','halloumi','brie','singles','le marechal','burrata','feta','parmesan','mozzarella','cheddar','gouda','provolone','velveeta'],
                             'juice, tea & coffee':['juice','naked','minute maid','drink','punch','lemonade','limeade','tea','kombucha','simply','coffee'],
                             'ice cream/novelty':['jello','chocolate','pudding','gelatin','moo tubes','candy','cookie dough','ice cream','novelty','whipped cream','whipped topping','cream puff','sorbet'],
                             'butter and spreads':['butter','parkay','pate','dressing','sour cream','mousse','margarine','ghee','shortening','spread','cream cheese','dip'],
                             'eggs':['eggs','egg whites','egg','egg yolks']
                             },
            
                    'produce':{'spices & herbs':['parsley','herbs','bay leaves','sage','yellow name','tarragon','tumeric','red oak leaf','white name','chicory','capers','okra','aloe','mint','basil','cilantro','italian herbs','garlic','ginger'],
                               'vegetables':['parsnip','yautia','kim chee','bok choy','collard green','sprout','artichoke','daikon','dill','pickles','malanga coco','kimchi','scallions','tempeh','veggie','vegetable','tofu','brussel sprout','brussels sprout','corn','artichokes','shallots','mushroom','asparagus','green beans','eggplant','broccoli','cabbage','fennel','carrot','celery','broccili','broccilini','cauliflower','seaweed','zucchini','potato','beet','yam','tomato','cucumber','onion','bean sprouts','squash','chard','leek','pepper','peas'],
                               'salad':['coleslaw','butter','caesar','butterhead','mix','radish','spring mix','salad','greens','kale','spinach','arugula','lettuce','romaine','rocket salad'],
                               'fruits':['papaya','tomatillo','olive','coconut','tangerines','honey dew','apples','pink lady','melon','pumpkin','mandarins','nectarines','prunes','dates','grapes','clementines','watermelon','banana','kiwi','kiwifruit','apple','grapefruit','avocado','watermelon','cantaloupe','mango','pomegranate','apricot','pear','plum','peach','peaches','lemon','lime','orange','cutie'],
                               'berries':['plantain','blueberry','cranberry','cranberries','gooseberries','platain','raspberries','blackberries','blueberries','strawberries','cherries','cranberries'],
                               'nuts and seeds' : ['pepitas','pecan','pistachios','filberts','raisins','almonds','cashews','cashew','almond','seeds','figs','nuts','nut']    
                             
                             },
                    
                    'meat':{'meat substitue':['meatless','tofu','beyond burger','impossible burger', 'beyond','plantbased','plant based'],                       
                            'poultry':['chicken','nugget','hen','tenders','turkey','duck','wings','drumsticks'],
                            'red meat':['franks','chorizo','pork','filet mignon','sauasge','beef','sirloin','meatball','carne','chuck','roast','bratwurst','wieners','wiener','hot dogs','hot dog','ribs','sausage','pork','burger patties','burger','burger patty','bacon','ham','lamb','veal','goat','venison','bison','boar','steak'],
                            'deli':['soppressata','salame','salami','prosciutto','salami','italian sausage','pepperoni']
                            },

                    'seafood':{'crusteceans':['shrimp','lobster','oysters','crab','prawns'],
                               'mollusks/bivalves':['scallops','octopus','squid','clams','mussels','abalone','snail'],
                               'fish':['perch','herring','sea bass','porgie','dab','atlantic','pollock','haddock','drumsticks','cod','salmon','fillet','fish','anchovies','porgy','tenders','branzini','sardines','sardine','mackerel','codfish','seafood','trout','mahi mahi','grouper','tuna','catfish','snapper','tilapia']
                        
                              }
                    }

In [93]:
print(subcat_key_words)

{'dairy': {'dairy substitutes': ['soy', 'dairy-free', 'almond', 'lactaid', 'coconut', 'oat', 'nut milk', 'silk', 'dairy free', 'plant based', 'plantbased', 'almondmilk', 'lactose free', 'lactosefree', 'soymilk', 'oatmilk', 'cashewmilk'], 'yogurt': ['yogurt', 'danimals', 'kefir', 'yoghurt', 'chobani', 'greek', 'smoothie'], 'milk products': ['half', 'smoothie', '2%', 'beverage', 'vanilla', 'cacao', 'mocha', 'nog', 'egg nog', 'milk', 'soymilk', 'buttermilk', 'half and half', 'half half', 'creamer', 'cream', 'heavy whipping cream', 'french vanilla'], 'cheese': ['cheese', 'piave', 'italian blend', 'grand cru', 'caramella', 'mozz', 'fromage', 'humboldt fog', 'saint angel', 'kefalograviera', 'ovalie cendree', 'chevre', 'pepper jack', 'cheddar', 'mahon', 'daffinois', 'queso', 'sargento', 'havarti', 'manchego', 'asiago ', 'mascarpone', 'halloumi', 'brie', 'singles', 'le marechal', 'burrata', 'feta', 'parmesan', 'mozzarella', 'cheddar', 'gouda', 'provolone', 'velveeta'], 'juice, tea & coffee': [

In [94]:
#function to find key words in the item name
def FindSubcategory(vec):
  clean_item_name = vec[27]
  Category = vec[2]
  item_name = vec[1].lower()
  # print(item_name)
  for cat in subcat_key_words:
    if cat == Category:
      for subcat in subcat_key_words[cat]:
        for key_word in subcat_key_words[cat][subcat]:
          #check if the any of the keywords for each subcategory are in the item name
          if key_word in clean_item_name or key_word+'s' in clean_item_name:
            #if yes set the the subcategory
            subcategory = subcat
            # print(subcategory)
            return subcategory


In [95]:
#apply function to get subcategory for each item
ny_df["subcategory"] = ny_df.apply(FindSubcategory,axis=1)

In [96]:
ny_df[['Item name','clean_item_name','Category','subcategory']].sample(10)

,Item name,clean_item_name,Category,subcategory
58843,"Green Way Organic Baby Spring Mix,11 OZ",green way organic baby spring mix,produce,salad
41140,"Simply Light Lemonade With Raspberry,52 FOZ",simply light lemonade with raspberry,dairy,"juice, tea & coffee"
99759,"Uncured Beef Hot Dog,12 OZ",uncured beef hot dog,meat,red meat
64104,"Lightlife Smart Ground,12 OZ",lightlife smart ground,produce,None
42338,"Foodtown Unsalted Whipped Butter,8 OZ",foodtown unsalted whipped butter,dairy,butter and spreads
66514,"Bolthouse Farms Protein Plus Protein Shake,15....",bolthouse farms protein plus protein shake,produce,None
42661,FIVE ACRE BUTTER MLK,five acre butter mlk,dairy,butter and spreads
52557,"Bubba Burgers 6Ct - Angus,32 OZ",bubba burgers 6ct angus,meat,red meat
2417,Grapes (Red - Organic),grapes,produce,fruits
45576,"Coffee-Mate Creamer, Peppermint Mocha,32 FOZ",coffeemate creamer peppermint mocha,dairy,milk products


In [97]:
#check for nulls for unique item names
x = ny_df.drop_duplicates(['clean_item_name','Category','subcategory'],keep= 'last')
x.count()
x= x[['Item name','clean_item_name','Category','subcategory']]
x[x['subcategory'].isnull()].sample(10)

,Item name,clean_item_name,Category,subcategory
35428,"Hebrew National Beef Knockwurst,12 OZ",hebrew national beef knockwurst,dairy,None
70690,"Everything Chopped Kit,11.54 OUNCE",everything chopped kit,produce,None
46246,"Sabrett Cocktail Franks, 12 Oz,12 OZ",sabrett cocktail franks,dairy,None
25870,"Smithfield Bacon,16 OZ",smithfield bacon,dairy,None
98258,"Sopressata Chub,6 OZ",sopressata chub,meat,None
74690,"Radicchio,1 HEAD",radicchio,produce,None
26824,"Carl Buddig Original Honey Roasted Turkey Box,...",carl buddig original honey roasted turkey box,dairy,None
67882,WATERCRESS,watercress,produce,None
67594,LA FLOR WHOLE ALLSPICE,la flor whole allspice,produce,None
30388,"Oscar Mayer Rotisserie Style Chicken Breast, 9...",oscar mayer rotisserie style chicken breast,dairy,None


In [98]:
x.count()

Item name          7671
clean_item_name    7671
Category           7671
subcategory        6941
dtype: int64

In [99]:
#check for nulls
x = ny_df.drop_duplicates(['clean_item_name','Category','subcategory'],keep= 'last')
x= x[['Item name','clean_item_name','Category','subcategory']]
#unique unclassified item names
x['subcategory'].isnull().sum()

730

In [100]:
#total number of uncategorized records for subcategory
ny_df['subcategory'].isnull().sum()

9828

Notice how a lot of these are not being classified correctly for subcategory because the are in the wrong overall category. Let's fix that:

In [101]:
#example
ny_df[['Item name','clean_item_name','Category','subcategory']].loc[[39280]]

,Item name,clean_item_name,Category,subcategory
39280,"Sugardale Jumbo Franks, 48 oz.,48 OZ",sugardale jumbo franks,dairy,None


In [102]:
#fix category on items that are incorrectly classifiied
for index,row in ny_df.iterrows():
  if row['subcategory']==None:
    for cat in subcat_key_words:
      for subcat in subcat_key_words[cat]:
            for key_word in subcat_key_words[cat][subcat]:
              #check if the any of the keywords for each subcategory are in the item name
              if key_word in row['clean_item_name'] or key_word+'s' in row['clean_item_name']:
                #if yes set the the subcategory
                # print('old:',row['Item name'],row['Category'],row['subcategory'])
                ny_df['subcategory'][index] = subcat
                ny_df['Category'][index] = cat
                # print('new:',row['Item name'],row['Category'],row['subcategory'])

<ipython-input-102-5efe4958ab5d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_df['subcategory'][index] = subcat
<ipython-input-102-5efe4958ab5d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_df['Category'][index] = cat


Now the item is in the correct category and subcategory

In [90]:
#check that category was corrected and subcategory was assigned
ny_df[['Item name','clean_item_name','Category','subcategory']].loc[[39280]]

,Item name,clean_item_name,Category,subcategory
39280,"Sugardale Jumbo Franks, 48 oz.,48 OZ",sugardale jumbo franks,dairy,None


In [103]:
#number of uncategorized records for subcategory
ny_df['subcategory'].isnull().sum()

5495

In [104]:
ny_df.count()

Unnamed: 0                102275
Item name                 102275
Category                  102275
Price                     102275
Unit                       97507
Unit Price                 98231
Store                     102275
Date                      102275
Product Number             91698
Sale Price                     0
Serving Size               34904
Serving Size Unit          34904
Calories                   36140
Total Fat (g)              36092
Saturated Fat (g)          35528
Trans Fat (g)              33984
Cholesterol (mg)           35108
Sodium (mg)                36124
Total Carbohydrate (g)     36128
Dietary Fiber (g)          33916
Total Sugars (g)           34572
Added Sugars (g)           23940
Protein (g)                36132
Borough                   102275
Address                   102275
Zip Code                  102275
Coordinates               102275
clean_item_name           102275
subcategory                96780
dtype: int64

In [105]:
#check for nulls
x = ny_df.drop_duplicates(['clean_item_name','Category','subcategory'],keep= 'last')
x= x[['Item name','clean_item_name','Category','subcategory']]
#unique unclassified item names
x.count()

Item name          7663
clean_item_name    7663
Category           7663
subcategory        7247
dtype: int64

In [106]:
x[x['subcategory'].isnull()].sample(10)

,Item name,clean_item_name,Category,subcategory
13600,"HILLSHIRE FARM Polska Kielbasa,14.0 OUNCE",hillshire farm polska kielbasa,meat,None
89650,Cana De Cabra,cana de cabra,dairy,None
71634,"Gourmet Medley,4.5 OZ",gourmet medley,produce,None
88258,"Sparkenhoe Red Leicester,0.19 LB",sparkenhoe red leicester,dairy,None
33772,"Bob Evans Liquid All Whites,32 OZ",bob evans liquid all whites,dairy,None
13388,"House Foods Organic, TofÃ¼,14.0 OUNCE",house foods organic tofã¼,produce,None
20156,"Creme Fraiche ,8.0 OUNCE",creme fraiche,dairy,None
91042,"Challerhocker,0.99 LB",challerhocker,dairy,None
21464,"Mama Mary's 7"" Gluten Free, Pizza Crust,7.0 OUNCE",mama marys 7 gluten free pizza crust,dairy,None
17592,"Macro Vegetarian Bento Box,14.0 OUNCE",macro vegetarian bento box,dairy,None


## Cheaper Alternatives Recommendations

In [107]:
#these should be inputs from the user
#the store that the user selected
chosen_store = "Aldi"
#filter the dataset for the selected store
filtered_store = ny_df.loc[(ny_df['Store'] == chosen_store)]
#This would be the list of items returned from Pony's model
grocery_list = filtered_store.sample(10)
grocery_list

,Unnamed: 0,Item name,Category,Price,Unit,Unit Price,Store,Date,Product Number,Sale Price,...,Dietary Fiber (g),Total Sugars (g),Added Sugars (g),Protein (g),Borough,Address,Zip Code,Coordinates,clean_item_name,subcategory
284,284,"Cheese (Gouda, Edam, or Smoked Gouda)",dairy,2.99,$/oz,0.43,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Manhattan,"517 E 117TH ST, NEW YORK, NY, 10035",10035,"(40.795251824401916, -73.94102084813397)",cheese,cheese
2560,2560,Oranges (Cara Cara),produce,4.89,$/lb,1.63,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Brooklyn,"3785 NOSTRAND AVE, BROOKLYN, NY, 11235",11235,"(40.58620176091237, -73.95168128857959)",oranges,fruits
1818,1818,Soft Cream Cheese Spread (Strawberry),dairy,1.49,$/oz,0.19,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Bronx,"5532 BROADWAY, BRONX, NY, 10463",10463,"(40.88099069767225, -73.90565087656735)",soft cream cheese spread,milk products
4501,4501,Country Style Pork Ribs ((sold per lb.)),meat,3.79,$/lb,3.79,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Brooklyn,"1628 FLATBUSH AVE, BROOKLYN, NY, 11210",11210,"(40.62741664063716, -73.94567882204815)",country style pork ribs,red meat
4565,4565,Fresh Family Pack Organic Grass Fed 85/15 Grou...,meat,4.49,$/lb,4.49,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Brooklyn,"528 GATEWAY DR, BROOKLYN, NY, 11239",11239,"(40.64924922731707, -73.8822178092683)",fresh family pack organic grass fed 8515 groun...,red meat
2402,2402,Grapes (Candy Heart (sold per lb.)),produce,3.99,$/lb,3.99,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Bronx,"1750 E GUN HILL RD STORE B, BRONX, NY, 10469",10469,"(40.869791141134634, -73.84825250886955)",grapes,fruits
3997,3997,Black Angus Beef (Country Style Ribs - sold pe...,meat,7.39,$/lb,7.39,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Brooklyn,"1628 FLATBUSH AVE, BROOKLYN, NY, 11210",11210,"(40.62741664063716, -73.94567882204815)",black angus beef,red meat
922,922,Greek Yogurt (Peach Non Fat),dairy,0.63,$/oz,0.12,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Brooklyn,"3785 NOSTRAND AVE, BROOKLYN, NY, 11235",11235,"(40.58620176091237, -73.95168128857959)",greek yogurt,yogurt
1518,1518,Organic Milk (Organic Reduced Fat - 2%),dairy,3.65,$/oz,0.06,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Queens,"61-11 JUNCTION BLVD, REGO PARK, NY, 11374",11374,"(40.72497322012315, -73.86190089826476)",organic milk,milk products
2083,2083,Yogurt Tubes (Gimmies Super Berry Rocket Yogur...,dairy,3.49,$/ct,0.35,Aldi,2022-10-30,NaN,NaN,...,NaN,NaN,NaN,NaN,Brooklyn,"3785 NOSTRAND AVE, BROOKLYN, NY, 11235",11235,"(40.58620176091237, -73.95168128857959)",yogurt tubes,yogurt


In [108]:
recommendations_dict = dict()
for index, row in grocery_list.iterrows():
    filtered_dataset = filtered_store.loc[(filtered_store['Price'] < row['Price']) & (filtered_store['Category'] == row['Category']) & (filtered_store['subcategory'] == row['subcategory'])].sort_values('Price')
    print(filtered_dataset)
    recommendation_info = filtered_dataset.iloc[1]
    recommendation_name = recommendation_info['Item name']
    recommendation_price = recommendation_info['Price']
    original_item = row['Item name']
    original_price = row['Price']
    print('Original Item:',original_item,', Original Price:',original_price,', Recommendation:',recommendation_name,', Recommendation Price:',recommendation_price)
    # return original_item,original_price,recommendation_name,recommendation_price


      Unnamed: 0                   Item name Category  Price  Unit  \
129          129  American Imitation Singles    dairy   0.99  $/ct   
134          134  American Imitation Singles    dairy   0.99  $/ct   
133          133  American Imitation Singles    dairy   0.99  $/ct   
132          132  American Imitation Singles    dairy   0.99  $/ct   
131          131  American Imitation Singles    dairy   0.99  $/ct   
...          ...                         ...      ...    ...   ...   
1360        1360         Mini Babybel Cheese    dairy   2.97  $/oz   
1359        1359         Mini Babybel Cheese    dairy   2.97  $/oz   
1367        1367         Mini Babybel Cheese    dairy   2.97  $/oz   
1365        1365         Mini Babybel Cheese    dairy   2.97  $/oz   
1362        1362         Mini Babybel Cheese    dairy   2.97  $/oz   

      Unit Price Store        Date  Product Number  Sale Price  ...  \
129         0.06  Aldi  2022-10-30             NaN         NaN  ...   
134         0.06 

## BERT Attempt Do not use

In [ ]:
df.Category.unique()

In [ ]:
df.Store.unique()

In [ ]:
unique_items = df['Item name'].unique()
for item in unique_items:
  print(item)

In [ ]:
#creating category dictionary... Category:subcategory:items
Subcategories = ["dairy":["yogurt","milk","cheese","creamer","sour cream","cream cheese"],"meat":["poultry":["chicken","turkey"],"beef","turkey","ham","hot dog","sausage","chorizo","burger","salami"],
                 "produce":["fruit":["strawberries","blueberries","orange","apple","bananna","blackberries"],"vegetables":["broccoli","asparagus","onions","spinach","califlower"]],"seafood":["fish":["salmon","tuna","cod","mahi mahi","flounder"]"crustaceans":["shrimp","lobster","mussles"]]

In [ ]:
vectorize_products_based_on_metadata("lobster")

BERT with pytorch for Subcategory Classification

In [ ]:
!pip install transformers
import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
# !pip freeze > requirement.txt

In [ ]:
#lowercase item name column
df['item_lower']=df['Item name'].str.lower()

#create vocabulary of unique items in the dataset
item_list =list(set(df['item_lower']))
# item_list

#assign each token and word in the item list to an index number
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(item_list):
   word_dict[w] = i + 4
   number_dict = {i: w for i, w in enumerate(word_dict)}
   vocab_size = len(word_dict)

In [ ]:
#unique item names in our dataset
word_dict

In [ ]:
class config:
    MODEL = 'bert-base-uncased'

    HIDDEN = 768

    MAX_LENGTH = 64
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 32

    EPOCHS = 2

    LR = (2e-5, 3e-5, 5e-5)
    EPS = 1e-8

    SEED = 23

In [ ]:
data = df

# print(data)

data = data.dropna()
# data = data.sample(n=20000)

encoder = LabelEncoder()
data["classes"] = encoder.fit_transform((data['Category']))
data = data[['item_lower', 'Category', 'classes']]


class TextClassificationDataset:

    def __init__(self,
                 texts,
                 labels):

        self.texts = texts
        self.labels = labels
        self.max_seq_length = config.MAX_LENGTH

        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            config.MODEL
        )

        self.sep_vid = self.tokenizer.vocab["[SEP]"]
        self.cls_vid = self.tokenizer.vocab["[CLS]"]
        self.pad_vid = self.tokenizer.vocab["[PAD]"]

    def __len__(self):

        return len(self.texts)

    def __getitem__(self, item):

        texts = str(self.texts[item])
        texts = " ".join(texts.split())
        inputs = self.tokenizer.encode_plus(
            texts,
            None,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            max_length=self.max_seq_length,
            return_token_type_ids=False,
            return_tensors="pt",
        )

        true_seq_length = len(inputs['input_ids'][0])
        pad_size = self.max_seq_length - true_seq_length
        pad_ids = torch.Tensor([self.pad_vid] * pad_size).long()
        ids = torch.cat((inputs['input_ids'][0], pad_ids))


        output_dict = {
            "ids": ids.flatten(),
            'attention_mask': inputs["attention_mask"][0].flatten(),
            'target' : torch.tensor(self.labels[item], dtype=torch.long)

        }

        

        return output_dict


SEED = 42

n_classes = len(data['classes'].unique())

train, val = train_test_split(
    data, test_size=0.30, random_state=SEED)


train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

In [ ]:
#create a function that formats the input sequences for three types of embeddings: token embedding, segment embedding, and position embedding.
#For instance, if the sentence is “The cat is walking. The dog is barking”, then the function should create a sequence in the following manner: “[CLS] the cat is walking [SEP] the dog is barking”. 
#After that, we convert everything to an index from the word dictionary. So the previous sentence would look something like “[1, 5, 7, 9, 10, 2, 5, 6, 9, 11]”. Keep in mind that 1 and 2 are [CLS] and [SEP] respectively. 

def make_batch():
   batch = []
   positive = negative = 0
   while positive != batch_size/2 or negative != batch_size/2:
       tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences)) 

       tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

       input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]


       segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

       # MASK LM
       n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # 15 % of tokens in one sentence
       cand_maked_pos = [i for i, token in enumerate(input_ids)
                         if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
       shuffle(cand_maked_pos)
       masked_tokens, masked_pos = [], []
       for pos in cand_maked_pos[:n_pred]:
           masked_pos.append(pos)
           masked_tokens.append(input_ids[pos])
           if random() < 0.8:  # 80%
               input_ids[pos] = word_dict['[MASK]'] # make mask
           elif random() < 0.5:  # 10%
               index = randint(0, vocab_size - 1) # random index in vocabulary
               input_ids[pos] = word_dict[number_dict[index]] # replace

       # Zero Paddings
       n_pad = maxlen - len(input_ids)
       input_ids.extend([0] * n_pad)
       segment_ids.extend([0] * n_pad)

       # Zero Padding (100% - 15%) tokens
       if max_pred > n_pred:
           n_pad = max_pred - n_pred
           masked_tokens.extend([0] * n_pad)
           masked_pos.extend([0] * n_pad)

       if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
           batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
           positive += 1
       elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
           batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
           negative += 1
   return batch

In [ ]:
print(torch.arange(30, dtype=torch.long).expand_as(input_ids))

In [ ]:
class Embedding(nn.Module):
   def __init__(self):
       super(Embedding, self).__init__()
       self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
       self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
       self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
       self.norm = nn.LayerNorm(d_model)

   def forward(self, x, seg):
       seq_len = x.size(1)
       pos = torch.arange(seq_len, dtype=torch.long)
       pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
       embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
       return self.norm(embedding)

In [ ]:
text = (
       'Hello, how are you? I am Romeo.\n'
       'Hello, Romeo My name is Juliet. Nice to meet you.\n'
       'Nice meet you too. How are you today?\n'
       'Great. My baseball team won the competition.\n'
       'Oh Congratulations, Juliet\n'
       'Thanks you Romeo'
   )
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split()))
word_list

In [ ]:
!pip install pyodbc

In [ ]:
import pyodbc
import requests

In [ ]:
api_key = 'Ex8jGMBYdKCghAclsHiGmcJULfdeL5kq8DJxMVdT'
# r = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search?query=apple&pageSize=2&api_key=Ex8jGMBYdKCghAclsHiGmcJULfdeL5kq8DJxMVdT')
!curl 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?limit=1&api_key=Ex8jGMBYdKCghAclsHiGmcJULfdeL5kq8DJxMVdT'

In [ ]:
#User Input Simulations
chosen_store_name = "Aldi"
# model_generated_grocery_list = #needs item name,nutritional info,price, category and store
sample_record = df.iloc[20]
sample_record["subcategory"] = "Cheese"
sample_record

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


Item name                 Blue Cheese Crumbles
Category                                 dairy
Price                                     2.49
Unit                                      $/oz
Unit Price                                 0.5
Store                                     Aldi
Date                                2022-10-30
Product Number                             NaN
Sale Price                                 NaN
Serving Size                               NaN
Serving Size Unit                          NaN
Calories                                   NaN
Total Fat (g)                              NaN
Saturated Fat (g)                          NaN
Trans Fat (g)                              NaN
Cholesterol (mg)                           NaN
Sodium (mg)                                NaN
Total Carbohydrate (g)                     NaN
Dietary Fiber (g)                          NaN
Total Sugars (g)                           NaN
Added Sugars (g)                           NaN
Protein (g)  

In [ ]:
#filter dataset for that store, category and less than that price
#for each item from the generate grocery list feed it into the model
#for item in list
# filtered dataset
# sort dataset
# recommendation = grab top 1
# display back to user
#next item

filtered_dataset = df.loc[(df['Store'] == sample_record['Store']) & (df['Price'] < sample_record['Price']) & (df['Category'] == sample_record['Category'])] #& (df[subcategory] == sample_record[subcategory])]
filtered_dataset.sort_values('Price')

,Item name,Category,Price,Unit,Unit Price,Store,Date,Product Number,Sale Price,Serving Size,...,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrate (g),Dietary Fiber (g),Total Sugars (g),Added Sugars (g),Protein (g)
153,Moo Tubes Assorted Varieties (Grape Bubble Gum...,dairy,0.00,$/oz,0.00,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,Yogurt (Vanilla),dairy,0.39,$/oz,0.07,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,Light Nonfat Yogurt (Blueberry or Vanilla),dairy,0.39,$/oz,0.07,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,Lowfat Yogurt (Original Strawberry or Strawber...,dairy,0.49,$/oz,0.08,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,Lowfat Yogurt (Original Peach or Cherry),dairy,0.49,$/oz,0.08,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,Coffee Creamer (Delightfully Pure Assorted Var...,dairy,2.29,$/oz,0.14,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,Ricotta Cheese (Whole Milk or Part Skim),dairy,2.29,$/oz,0.15,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,Organic Deli Sliced Cheese (Colby Jack),dairy,2.35,$/oz,0.39,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Vegan Cream Cheese (Plain) [Vegan],dairy,2.39,$/oz,0.30,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from scipy.sparse.construct import random
#creating sample grocery list
sample_grocery_list = df.loc[df['Store'] == "Aldi"].sample(n=5,random_state=1)
sample_grocery_list


,Item name,Category,Price,Unit,Unit Price,Store,Date,Product Number,Sale Price,Serving Size,...,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrate (g),Dietary Fiber (g),Total Sugars (g),Added Sugars (g),Protein (g)
600,Salami Collection,meat,6.49,$/oz,0.54,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Butter (Butter Quarters - Salted or Unsalted),dairy,3.98,$/oz,0.25,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,Two Good Greek Yogurt (Strawberry or Vanilla L...,dairy,1.24,$/oz,0.23,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
677,White Shrimp,seafood,5.49,$/oz,0.55,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Cream Cheese (Original),dairy,1.59,$/oz,0.20,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sample_grocery_list.reset_index()
sample_grocery_list.loc[600,"subcategory"]="Cold Cuts"
sample_grocery_list.loc[23,"subcategory"]="Butter"
sample_grocery_list.loc[214,"subcategory"]="Yogurt"
sample_grocery_list.loc[677,"subcategory"]="Crusteaceans"
sample_grocery_list.loc[47,"subcategory"]="Cream Cheese"
sample_grocery_list


,Item name,Category,Price,Unit,Unit Price,Store,Date,Product Number,Sale Price,Serving Size,...,Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrate (g),Dietary Fiber (g),Total Sugars (g),Added Sugars (g),Protein (g),subcategory
600,Salami Collection,meat,6.49,$/oz,0.54,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cold Cuts
23,Butter (Butter Quarters - Salted or Unsalted),dairy,3.98,$/oz,0.25,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Butter
214,Two Good Greek Yogurt (Strawberry or Vanilla L...,dairy,1.24,$/oz,0.23,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yogurt
677,White Shrimp,seafood,5.49,$/oz,0.55,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crusteaceans
47,Cream Cheese (Original),dairy,1.59,$/oz,0.20,Aldi,2022-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cream Cheese


In [ ]:
sample_grocery_list.reset_index()
chosen_store = "Aldi"
filtered_store = df.loc[(df['Store'] == chosen_store)]
recommendations_dict = dict()
for index, row in sample_grocery_list.iterrows():
    filtered_dataset = filtered_store.loc[(filtered_store['Price'] < row['Price']) & (filtered_store['Category'] == row['Category'])].sort_values('Price') #& (df[subcategory] == sample_record[subcategory])]
    # print(filtered_dataset.iloc[1])
    recommendation = filtered_dataset.iloc[1]
    recommendations_dict[index]=recommendation

# recommendations_dict

In [ ]:
print(recommendations_dict)

{600: Item name                 Chicken Leg Quarters (Sold in a 10 lb. Bag)
Category                                                         meat
Price                                                            0.65
Unit                                                             $/lb
Unit Price                                                       0.65
Store                                                            Aldi
Date                                                       2022-10-30
Product Number                                                    NaN
Sale Price                                                        NaN
Serving Size                                                      NaN
Serving Size Unit                                                 NaN
Calories                                                          NaN
Total Fat (g)                                                     NaN
Saturated Fat (g)                                                 NaN
Trans Fat (g) 